<a href="https://colab.research.google.com/github/BrianSandiford/Technical-Assignment---Barbados/blob/main/health_registry_eda_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/health_registry.csv')
df.head()


,facility_id,facility_name,facility_type,capacity,region,licence_issue_date,inspection_date,gps_location,remarks
0,HF-0000,Burgess-Ingram Medical Center,Community Health Ctr.,250 beds,St.Lucy,04-01-21,08 Mar 2024,"13.08576, -58.75331",NaN
1,033477,Collins Inc,Hosp.,NaN,St.Peter,28 Mar 2024,17-04-23,POINT(-58.84001 12.87196),Good
2,#2,Butler-Leon Clinic (St.),Hospital,117Beds,St. James Parish,2016-02-18,30/12/22,"-58.82307, 13.00952",Good
3,2,Butler-Leon Clinic (St.),Hospital,117Beds,St. James Parish,2016-02-18,30/12/22,"-58.82307, 13.00952",Good
4,HF-0003,"Powell, Morales and Butler Infirmary",Health Ctr.,179 capacity,ST. LUCY,November 21 2020,07-03-24,"12.77293, -59.04279",Follow-up 2023


In [31]:
# Clean facility_id
df['facility_id'] = df['facility_id'].astype(str).str.strip().str.upper()

In [32]:
# Normalize capacity field
def extract_capacity(value):
    # Return None if the value is missing
    if pd.isnull(value):
      return None

     # Convert the value to a string
    text =str(value)

    # Keep only the digits
    number = ''
    for char in text:
        if char.isdigit():
            number += char

    # Return the number as an integer, or None if no digits were found
    if number:
        return int(number)
    else:
       return None

df['capacity'] = df['capacity'].apply(extract_capacity)



In [33]:
# Standardizing Dates

def robust_parse_date(date_str):
    if pd.isna(date_str):
        return pd.NaT, True

    # Convert to string and strip whitespace
    str_date = str(date_str).strip()

    # List of formats to try
    formats = [
        "%Y-%m-%d", "%Y/%m/%d",      # ISO formats
        "%d-%m-%y", "%d/%m/%y",      # Short year (e.g., 15-04-23)
        "%d-%m-%Y", "%d/%m/%Y",      # Full day-first
        "%B %d %Y"                   # Natural language: September 19 2022
    ]

    for fmt in formats:
        try:
            return datetime.strptime(str_date, fmt), False
        except ValueError:
            continue

    return pd.NaT, True

df[['licence_issue_date_clean', 'licence_issue_invalid']] = df['licence_issue_date'].apply(
    lambda x: pd.Series(robust_parse_date(x))
)


In [34]:
df[['inspection_date_clean', 'inspection_date_invalid']] = df['inspection_date'].apply(
    lambda x: pd.Series(robust_parse_date(x))
)


In [39]:

def extract_coords(value):
    if pd.isna(value):
        return pd.Series([None, None])

    str_val = str(value).strip()

    # Try POINT format
    if 'POINT' in str_val:
        try:
            coords = str_val.replace('POINT(', '').replace(')', '').split()
            return pd.Series([float(coords[1]), float(coords[0])])
        except:
            return pd.Series([None, None])

    # Try comma-separated format
    if ',' in str_val:
        try:
            parts = [float(p.strip()) for p in str_val.split(',')]
            if len(parts) == 2:
                return pd.Series(parts)
        except:
            pass  # Move on to DMS check

    # Try DMS format (handle both clean and garbled)
    try:
        if 'â' in str_val or 'Â' in str_val:  # garbage characters detected
            str_val = str_val.encode('latin1').decode('utf8')
        match = re.findall(r"(\d+)°(\d+)′(\d+)″([NSWE])", str_val)
        if len(match) == 2:
            def convert(d, m, s, direction):
                decimal = int(d) + int(m)/60 + int(s)/3600
                return -decimal if direction in ['S', 'W'] else decimal

            lat = convert(*match[0])
            lon = convert(*match[1])
            return pd.Series([lat, lon])
    except:
        pass

    return pd.Series([None, None])





In [40]:
df[['latitude', 'longitude']] = df['gps_location'].apply(extract_coords)

In [41]:
df[['gps_location', 'latitude', 'longitude']].head(20)

,gps_location,latitude,longitude
0,"13.08576, -58.75331",13.085760,-58.753310
1,POINT(-58.84001 12.87196),12.871960,-58.840010
2,"-58.82307, 13.00952",-58.823070,13.009520
3,"-58.82307, 13.00952",-58.823070,13.009520
4,"12.77293, -59.04279",12.772930,-59.042790
5,"12.77293, -59.04279",12.772930,-59.042790
6,13°12′43″N 58°51′50″W,13.211944,-58.863889
7,13°2′6″N 59°0′38″W,13.035000,-59.010556
8,13°2′6″N 59°0′38″W,13.035000,-59.010556
9,13°14′8″N 59°2′1″W,13.235556,-59.033611


In [47]:
# Normalize region names
df['region_clean'] = (
    df['region']
    .astype(str)
    .str.strip()
    .str.replace(r'\bparish\b', '', case=False, regex=True)  # removes 'Parish' in any case
    .str.strip()
    .str.title()
)
df[['region', 'region_clean']].drop_duplicates().head()


,region,region_clean
0,St.Lucy,St.Lucy
1,St.Peter,St.Peter
2,St. James Parish,St. James
4,ST. LUCY,St. Lucy
6,St Andrew,St Andrew


In [48]:
df_clean = df.drop_duplicates()
df_clean.shape

(11185, 16)

In [50]:

# Optional: list of known clean region names to validate against
known_regions = {
    "St. Michael", "St. Joseph", "St. Lucy", "St. Andrew",
    "St. Peter", "St. James", "St. George", "St. Philip",
    "St. Thomas", "Christ Church"
}

def clean_region(value):
    if pd.isna(value):
        return None

    text = str(value).strip()

    # Remove the word "Parish" (any capitalization)
    text = re.sub(r'\bparish\b', '', text, flags=re.IGNORECASE).strip()

    # Try forward-cleaned version
    forward = text.title()

    # Try reversed version
    reversed_text = text[::-1].strip()
    reversed_cleaned = re.sub(r'\bparish\b', '', reversed_text, flags=re.IGNORECASE).strip().title()

    # If reversed_cleaned is a known valid region, return it
    if reversed_cleaned in known_regions:
        return reversed_cleaned

    # Otherwise return forward-cleaned
    return forward


In [51]:
df['region_clean'] = df['region'].apply(clean_region)


In [53]:
df.head(30)

,facility_id,facility_name,facility_type,capacity,region,licence_issue_date,inspection_date,gps_location,remarks,licence_issue_date_clean,licence_issue_invalid,inspection_date_clean,inspection_date_invalid,latitude,longitude,region_clean
0,HF-0000,Burgess-Ingram Medical Center,Community Health Ctr.,250.0,St.Lucy,04-01-21,08 Mar 2024,"13.08576, -58.75331",NaN,2021-01-04,False,NaT,True,13.085760,-58.753310,St.Lucy
1,033477,Collins Inc,Hosp.,NaN,St.Peter,28 Mar 2024,17-04-23,POINT(-58.84001 12.87196),Good,NaT,True,2023-04-17,False,12.871960,-58.840010,St.Peter
2,#2,Butler-Leon Clinic (St.),Hospital,117.0,St. James Parish,2016-02-18,30/12/22,"-58.82307, 13.00952",Good,2016-02-18,False,2022-12-30,False,-58.823070,13.009520,St. James
3,2,Butler-Leon Clinic (St.),Hospital,117.0,St. James Parish,2016-02-18,30/12/22,"-58.82307, 13.00952",Good,2016-02-18,False,2022-12-30,False,-58.823070,13.009520,St. James
4,HF-0003,"Powell, Morales and Butler Infirmary",Health Ctr.,179.0,ST. LUCY,November 21 2020,07-03-24,"12.77293, -59.04279",Follow-up 2023,2020-11-21,False,2024-03-07,False,12.772930,-59.042790,St. Lucy
5,HF-0003,"Powell, Morales and Butler Infirmary",Health Ctr.,179.0,ST. LUCY,November 21 2020,07-03-24,"12.77293, -59.04279",Follow-up 2023,2020-11-21,False,2024-03-07,False,12.772930,-59.042790,St. Lucy
6,00004,"Williams, Woods and Bradley Polyclinic",Hospital,NaN,St Andrew,17-04-21,January 31 2024,13°12′43″N 58°51′50″W,NaN,2021-04-17,False,2024-01-31,False,13.211944,-58.863889,St Andrew
7,NAN,May and Sons Medical Center (St),Polyclinic 🏥,NaN,St Peter,20160217,2018-01-01,13°2′6″N 59°0′38″W,Follow-up 2023,NaT,True,2018-01-01,False,13.035000,-59.010556,St Peter
8,NAN,May and Sons Medical Center (St),Polyclinic 🏥,NaN,St Peter,20160217,2018-01-01,13°2′6″N 59°0′38″W,Follow-up 2023,NaT,True,2018-01-01,False,13.035000,-59.010556,St Peter
9,00006,"Herrera,-Taylor-and-Sanchez (St.Andrew)",Health Centre,NaN,St.Andrew,06/11/21,22 Mar 2018,13°14′8″N 59°2′1″W,-,2021-11-06,False,NaT,True,13.235556,-59.033611,St.Andrew


In [54]:
# remove duplicates
df_clean = df.drop_duplicates()

In [57]:
# Keep only selected columns
df_export = df_clean[[
    'facility_id','facility_name','facility_type','region_clean', 'latitude', 'longitude',
    'capacity', 'licence_issue_date_clean', 'inspection_date_clean','remarks'
]]

# Save the cleaned version to CSV
df_export.to_csv("/content/drive/MyDrive/cleaned_health_registry.csv", index=False)